In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import swifter

In [3]:
recipes = pd.read_csv('./recipes.csv')
requests = pd.read_csv('./requests.csv')
reviews = pd.read_csv('./reviews.csv')
diet = pd.read_csv('./diet.csv')
classify_diet = pd.read_csv('./classify_diet.csv', sep=';')

/tmp/ipykernel_2031/3137903501.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv('./reviews.csv')


In [4]:
recipes.head()

,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
1,365718,Cashew-chutney Rice,3600,600,Other,"c(""\""1\"""", ""\""3/4\"""", ""\""6\"""", ""\""5\"""", ""\""2\""...","c(""\""celery\"""", ""\""onion\"""", ""\""butter\"""", ""\""...",370.8,17.5,7.2,22.9,553.3,44.3,1.6,2.2,9.4,8.0,NaN
2,141757,Copycat Taco Bell Nacho Fries BellGrande,3600,2700,Other,"c(""\""3\"""", ""\""1/2\"""", ""\""1\"""", ""\""1\"""", ""\""3\""...","c(""\""Copycat Taco Bell Seasoned Beef\"""", ""\""ye...",377.6,20.9,10.5,45.7,1501.8,36.6,3.8,6.1,12.9,8.0,NaN
3,280351,Slow Cooker Jalapeno Cheddar Cheese Soup,18000,1800,Other,"c(""\""2\"""", ""\""1\"""", ""\""2\"""", ""\""2\"""", ""\""1\"""",...","c(""\""unsalted butter\"""", ""\""yellow onion\"""", ""...",282.8,16.5,10.3,50.5,630.2,22.8,2.3,2.7,11.7,6.0,NaN
4,180505,Cool & Crisp Citrus Chiffon Pie,3600,1800,Other,"c(""\""1\"""", ""\""1/4\"""", ""\""1/2\"""", ""\""1/2\"""", ""\...","c(""\""unflavored gelatin\"""", ""\""water\"""", ""\""su...",257.5,8.6,2.4,110.7,160.9,39.8,0.4,30.2,6.3,6.0,NaN


In [9]:
requests.head()

,AuthorId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1799.950949,0.0,Indifferent,0,0,0
1,437641B,365718,4201.820980,0.0,Yes,0,Indifferent,1
2,1803340263D,141757,6299.861496,0.0,Indifferent,1,Indifferent,0
3,854048B,280351,19801.365796,0.0,Yes,1,0,1
4,2277685E,180505,5400.093457,0.0,Indifferent,0,0,0


In [8]:
reviews.head()

,AuthorId,RecipeId,Rating,Like,TestSetId
0,2492191A,33671,2.0,NaN,1.0
1,2002019979A,92647,2.0,NaN,2.0
2,408594E,161770,NaN,NaN,3.0
3,2001625557E,108231,2.0,NaN,4.0
4,2001427116E,71109,NaN,NaN,5.0


In [7]:
diet.head()

,AuthorId,Diet,Age
0,10000120E,Vegetarian,46
1,1000014D,Vegan,18
2,1000015A,Vegetarian,58
3,1000016E,Vegetarian,32
4,1000027E,Vegan,61


In [5]:
recipes['RecipeIngredientParts'] = recipes['RecipeIngredientParts'].apply(lambda x: x.replace('\\','').replace('"','').replace('c(', '').replace(')','').replace('(',''))
recipes['RecipeIngredientQuantities'] = recipes['RecipeIngredientQuantities'].apply(lambda x: x.replace('\\','').replace('"','').replace('c(', '').replace(')','').replace('(',''))

def func(x):
    res = []
    for y in x.split(','):
        y = y.strip().lower().replace('-',' ')
        found = classify_diet.loc[classify_diet['Ingredient'] == y]['Diet'].values
        if len(found) == 0:
            print(y)
            continue
        else:
            res.append(found[0])
    return res

# add meal type
classify_diet['Ingredient'] = classify_diet['Ingredient'].str.lower().str.strip().str.replace('(', '').str.replace(')', '').replace('-',' ')
recipes['RecipeIngredientParts'] = recipes['RecipeIngredientParts'].str.lower().str.strip().replace('-',' ')

#classify_diet
print(len(recipes))
recipes['DietIngredient'] = recipes['RecipeIngredientParts'].swifter.apply(func)

recipes['DietIngredient']

75604


Pandas Apply:   0%|          | 0/75604 [00:00<?, ?it/s]

fresh thyme leave
zatarians gumbo base mix
oak grove smoke house gumbo base mix
fresh thyme leave
chicken bouillon granule
fresh thyme leave
chicken bouillon granule
frisee
flax seed
frozen blackberrie
flax seed
fresh thyme leave
fresh thyme leave
lowfat mozzarella cheese


In [99]:
# join recipes and requests
joined = recipes.merge(reviews, on='RecipeId', how='inner')

# join with diet
joined = joined.merge(diet, on='AuthorId', how='inner')

# join with requests
all = joined.merge(requests, on=['RecipeId', 'AuthorId'], how='inner')

In [43]:
all.head()

,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,...,Like,TestSetId,Diet,Age,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,...,NaN,1548.0,Omnivore,34,1800.709345,0.0,Indifferent,0,0,1
1,243768,Pepper Jelly Chicken Roll Ups,480,0,Other,"c(""\""1/4\"""", ""\""4\"""", ""\""1/4\"""", ""\""2\"""")","c(""\""skinless chicken breasts\"""", ""\""olive oil...",365.2,5.6,1.2,...,NaN,1547.0,Omnivore,34,478.844112,0.0,Yes,1,Indifferent,0
2,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,...,NaN,2069.0,Omnivore,22,1797.825626,0.0,Yes,0,Indifferent,1
3,32618,Sharon's Best Rib-Eye Steaks,600,4200,Other,"c(""\""4\"""", ""\""1\"""", ""\""1\"""", ""\""2\"""", ""\""2\"""",...","c(""\""olive oil\"""", ""\""paprika\"""", ""\""garlic cl...",21.7,1.9,0.3,...,NaN,2070.0,Omnivore,22,4798.727142,1.0,Indifferent,1,0,0
4,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,...,NaN,2152.0,Omnivore,44,1800.440504,1.0,Indifferent,0,0,0


In [100]:
mapping = {
    'RecipeCategory': 'string',
    'RecipeIngredientQuantities': 'string',
    'RecipeIngredientParts': 'string',
    'Name': 'string',
    'AuthorId': 'string',
    'Like': 'bool',
    'Diet': 'string',
    'HighCalories': 'bool',
    'HighProtein': 'string',
    'LowFat': 'bool',
    'LowSugar': 'string',
    'HighFiber': 'bool'
}
df = all.astype(mapping)

In [68]:
df.head()

,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,...,Like,TestSetId,Diet,Age,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"6, 2, 1 1/2, 1/4, 1/2, 4, 1 1/2, 1 1/2, 5, 12","hazelnuts, broccoli florets, fresh parsley lea...",241.3,10.1,1.2,...,True,1548.0,Omnivore,34,1800.709345,False,Indifferent,False,0,True
1,243768,Pepper Jelly Chicken Roll Ups,480,0,Other,"1/4, 4, 1/4, 2","skinless chicken breasts, olive oil",365.2,5.6,1.2,...,True,1547.0,Omnivore,34,478.844112,False,Yes,True,Indifferent,False
2,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"6, 2, 1 1/2, 1/4, 1/2, 4, 1 1/2, 1 1/2, 5, 12","hazelnuts, broccoli florets, fresh parsley lea...",241.3,10.1,1.2,...,True,2069.0,Omnivore,22,1797.825626,False,Yes,False,Indifferent,True
3,32618,Sharon's Best Rib-Eye Steaks,600,4200,Other,"4, 1, 1, 2, 2, 2, 1 1/2, 1, 1, 1","olive oil, paprika, garlic cloves, dried thyme...",21.7,1.9,0.3,...,True,2070.0,Omnivore,22,4798.727142,True,Indifferent,True,0,False
4,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"6, 2, 1 1/2, 1/4, 1/2, 4, 1 1/2, 1 1/2, 5, 12","hazelnuts, broccoli florets, fresh parsley lea...",241.3,10.1,1.2,...,True,2152.0,Omnivore,44,1800.440504,True,Indifferent,False,0,False


In [9]:
all_ing = df['RecipeIngredientParts'].str.split(',').explode().str.strip().unique()
for ing in all_ing:
    print(ing)
print(len(all_ing))

Omnivore
Vegan
Vegetarian
3


In [81]:
all_ing = df['LowSugar'].unique()
for ing in all_ing:
    print(ing)
print(len(all_ing))

0
Indifferent
2


0                          hazelnuts
1                   broccoli florets
2               fresh parsley leaves
3                          olive oil
4                          mint leaf
                    ...             
4932          12-inch flour tortilla
4933                      veal broth
4934    classic buttercream frosting
4935                  canned lychees
4936       vegetarian cheddar cheese
Name: Ingredient, Length: 4937, dtype: object

In [ ]:
# add feature Rated
df['Rated'] = df['Rating'].apply(lambda x: True if x > 0 else False)

# HighProtein and LowSugar have only two values
mapping = {
    'HighProtein': 'bool',
    'LowSugar': 'bool',
}
df = df.astype(mapping)

In [35]:
print(df.groupby("Like").size())

Like
False    84503
True     55692
dtype: int64


In [ ]:
def boxplot(column):
    sns.boxplot(data=df,x=df[f"{column}"])
    plt.show()

cols = ['Age', 'CookTime', 'PrepTime', 'Calories', 'FatContent', 'SaturatedFatContent', 'CarbohydrateContent', 'FiberContent', 
        'SugarContent', 'ProteinContent', 'SodiumContent', 'CholesterolContent']

for col in cols:
    boxplot(col)

We can see some outlier which we will remove

In [7]:
# takes around 3 minutes
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

numeric_cols = ['Age', 'CookTime', 'PrepTime', 'Calories', 'FatContent', 'SaturatedFatContent', 'CarbohydrateContent', 'FiberContent', 
        'SugarContent', 'ProteinContent', 'SodiumContent', 'CholesterolContent']
df_lof = df[numeric_cols]

scaler = StandardScaler()
df_lof = scaler.fit_transform(df_lof)

clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(df_lof)
X_scores = clf.negative_outlier_factor_

In [83]:
pd.set_option('display.max_columns', None)
df_filtered = df[X_scores < -3.5]
df_filtered

,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,AuthorId,Rating,Like,TestSetId,Diet,Age,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
177,160180,Apricot Honey Syrup,2073600,600,Other,"1/2, 3, 3, 4, 2","honey, sugar, water",6736.0,1.2,0.0,0.0,72.9,1780.3,18.6,1756.0,10.8,NaN,1 quart,1073154D,2.0,False,NaN,Vegetarian,59,2.074200e+06,False,Indifferent,False,0,False
2397,184577,Crock Pot Barbecued BBQ Ribs,28800,1500,Other,"2, 1, 1","salt, pepper, onion, barbecue sauce",2075.5,159.9,51.3,545.3,1610.8,45.3,0.8,32.6,105.4,NaN,NaN,751444B,NaN,False,NaN,Vegetarian,18,3.029964e+04,False,Indifferent,False,0,False
3658,328309,Crock Pot Wild Rice Mix,10800,300,Other,"2, 4",water,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,8.0,NaN,104168A,2.0,True,30964.0,Vegetarian,19,1.110044e+04,False,Indifferent,False,0,True
4924,113918,Pineapple-Stuffed Cornish Game Hens,5400,2400,Other,"2, 1/2, 1, 3, 1, 1/2, 2/3, 1/4, 2, 2, 2, 1, 1","salt, crushed pineapple, celery rib, flaked co...",1583.7,88.1,49.4,678.7,1833.6,75.3,3.4,40.5,119.6,2.0,NaN,1137048B,2.0,True,NaN,Vegetarian,37,7.799589e+03,True,Indifferent,False,0,False
5464,69209,Israeli Roast Duck (Or Chicken),5400,1200,Other,"1, 1/2, 1, 3/4, 1, 1, 1, 3/4, 1, 1","ducklings, orange, light brown sugar, onion, c...",1242.2,62.3,16.8,773.1,557.2,20.8,1.4,14.4,141.3,4.0,NaN,2001774407A,2.0,False,NaN,Vegetarian,34,6.598655e+03,True,Yes,False,0,False
6626,109764,Cornish Game Hens With Creamy Orange Sauce,4500,600,Other,"2, 2, 1/4, 2, 2, 1/4, 1, 1/2","salt, butter, orange marmalade, butter, flour,...",1264.4,46.5,22.2,594.6,676.7,91.4,1.1,72.0,119.2,NaN,2 game hens,929391D,NaN,True,NaN,Vegetarian,63,5.100160e+03,True,Indifferent,False,0,False
9079,504695,Peach-Glazed Spareribs,3600,600,Other,"1, 1, 1/3, 1/4, 1/4, 2, 1, 1/4, 3 -4","garlic clove, soy sauce, honey, brown sugar, s...",2184.3,156.1,52.8,548.5,2219.5,55.3,2.3,51.4,136.1,NaN,NaN,2000507635A,NaN,True,40366.0,Vegetarian,43,4.199222e+03,False,Indifferent,False,0,False
11864,124978,Avocado Dip,900,600,Other,"1, 1, 150, 25, 5, 5, 1","avocado, cardamom, raisins, cashews, butter, ghee",9414.6,15.4,4.2,10.9,14.3,2397.7,5.3,2387.5,2.3,NaN,2 cups,2000333540A,NaN,True,35943.0,Vegetarian,23,1.498642e+03,False,Yes,True,Indifferent,True
15888,160180,Apricot Honey Syrup,2073600,600,Other,"1/2, 3, 3, 4, 2","honey, sugar, water",6736.0,1.2,0.0,0.0,72.9,1780.3,18.6,1756.0,10.8,NaN,1 quart,1222368B,2.0,True,NaN,Vegetarian,31,2.074202e+06,True,Indifferent,True,0,True
16482,113162,Salt-Baked Sea Bass,1200,240,Other,"2, 1 3/4, 2",salt,17.2,0.1,0.0,0.0,339187.3,0.2,0.0,0.2,3.6,2.0,NaN,2000271130C,NaN,False,NaN,Vegetarian,51,1.438459e+03,False,Indifferent,False,0,False


In [ ]:
train_all = all[all['TestSetId'].isna()]
test = all[all['TestSetId'].notna()]